In [ ]:
# Basic imports
 
import torch 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 

In [ ]:
# import the data 
data = pd.read_csv("../input/predict-closed-questions-on-stack-overflow/train-sample.csv")
data.head()

In [ ]:
data.OpenStatus.value_counts()

In [ ]:
# Let's take 'TITLE' & 'BODYMARKDOWN' & OpenStatus Columns 
data_train = data[['Title', 'BodyMarkdown', 'OpenStatus']]
data_train.head()

In [ ]:
data_train.shape  # it's tooo big, let's take just 20 k rows. 

In [ ]:
# Sample 80k randomly! 
data_train = data_train.sample(80000, random_state = 234)
data_train.head()

In [ ]:
%%capture
!pip install texthero  # For text pre processing 

In [ ]:
# Let's pre process the data using library called "Text_hero"
import texthero as hero 

data_train['Title'] = hero.clean(data_train['Title'])
data_train['BodyMarkdown'] = hero.clean(data_train['BodyMarkdown'])

```Python 
hero.clear(df['something'])  

# It does many things, They are: 
fillna()
lowercase()
remove_digits()
remove_punctuation()
remove_diacritics()
remove_stopwords()
remove_whitespace()
```

In [ ]:
# Let's see the data now! 
data_train.head()  # look how clean our text is:) 

In [ ]:
# Let's change the target to numbers: 
data_train.OpenStatus.value_counts()

In [ ]:
data_train['OpenStatus']= data_train['OpenStatus'].map({'open': 0, 'not a real question': 1, 'off topic': 2, 'not constructive': 3, 'too localized': 4}) 
data_train.head()

In [ ]:
# Let's seperate x and y: 
x = data_train['Title'] + ' '+ data_train['BodyMarkdown']
y = data_train['OpenStatus']

# xtrain, ytrian 
from sklearn.model_selection import train_test_split 
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.3, random_state = 203)

```Python 
Let us go from basic to Advance modeling :) 
```

In [ ]:
# Let's covvert words to numbers using TF-IDF 
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features = 10000)  # it contains only 10k features (fixed!)

xtrain_tfidf = vectorizer.fit_transform(xtrain).toarray()  # converting words to numbers for train data 
xtest_tfidf = vectorizer.transform(xtest).toarray()        # converting words to numbers for test data 

In [ ]:
# Naive Bayes 
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

clf.fit(xtrain_tfidf, ytrain)

predicted_naive = clf.predict(xtest_tfidf)

In [ ]:
# Metrics :) 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix 

print('Accuracy Score \n',accuracy_score(predicted_naive, ytest))
print('Confusion Matrix \n', confusion_matrix(predicted_naive, ytest))
print('Classification Report \n', classification_report(predicted_naive, ytest))

In [ ]:
# MLP classifier 
from sklearn.neural_network import MLPClassifier

mlp_cv=MLPClassifier(early_stopping=True, verbose=2)
mlp_cv.fit(xtrain_tfidf, ytrain)

In [ ]:
# predict :) 
predicted_mlp = mlp_cv.predict(xtest_tfidf)

In [ ]:
# Metrics :) 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix 

def metrics(predicted): 
    predicted_naive = predicted 
    print('Accuracy Score \n',accuracy_score(predicted_naive, ytest))
    print('Confusion Matrix \n', confusion_matrix(predicted_naive, ytest))
    print('Classification Report \n', classification_report(predicted_naive, ytest))

In [ ]:
metrics(predicted_mlp)